In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from google.colab import files
filename = "data.xlsx"
df = pd.read_excel(filename)

In [ ]:
import pandas as pd
import numpy as np
import hashlib
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from pandas.tseries.offsets import MonthEnd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools
import statsmodels.tsa.api as tsa


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from google.colab import files
import hashlib

# Function to process the Excel file
def process_excel(filepath):
    """
    Load the dataset from the specified filepath, select certain columns,
    anonymize the 'Resource' column, and ensure numeric columns are properly formatted.
    Additionally, parse the 'Work Date' column into datetime.

    Parameters:
        filepath (str): The path to the Excel file containing the dataset.

    Returns:
        pd.DataFrame: A pandas DataFrame containing the cleaned and anonymized data.
    """
    # Columns to use
    use_columns = [
        'Resource', 'Work Date', 'Project Number', 'Activity Type',
        'Company Number', 'Non Billable Hours', 'Billable Hours Worked',
        'Department', 'Totaal uren'
    ]

    # Load the data
    df = pd.read_excel(
        filepath,
        usecols=use_columns
    )

    # Convert 'Work Date' to datetime format
    df['Work Date'] = pd.to_datetime(df['Work Date'], dayfirst=True)

    # Convert numeric fields that use commas for decimals
    df['Non Billable Hours'] = pd.to_numeric(df['Non Billable Hours'].astype(str).str.replace(',', '.'), errors='coerce')
    df['Billable Hours Worked'] = pd.to_numeric(df['Billable Hours Worked'].astype(str).str.replace(',', '.'), errors='coerce')
    df['Totaal uren'] = pd.to_numeric(df['Totaal uren'].astype(str).str.replace(',', '.'), errors='coerce')

    # Anonymize the 'Resource' column using a hash function
    def anonymize_resource(resource_name):
        # Create a hash object with SHA-256
        hash_obj = hashlib.sha256()
        # Update the hash object with the resource name, encoded to bytes
        hash_obj.update(resource_name.encode())
        # Return the hexadecimal digest of the hash
        return hash_obj.hexdigest()

    # Apply anonymization to the 'Resource' column
    df['Resource'] = df['Resource'].apply(anonymize_resource)

    return df

# Step 1: Upload your Excel file
uploaded = files.upload()

# Process the uploaded Excel file
filepath = next(iter(uploaded.keys()))
df = process_excel(filepath)

In [ ]:
df.info()
summary_statatistics = df.describe()

summary_statatistics

In [ ]:
def summarize_categories(df):
    """
    Summarize the DataFrame by categories, focusing on department and activity type.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the cleaned and anonymized data.

    Returns:
        pd.DataFrame: A DataFrame with summarized categorical data.
    """
    # Grouping by 'Department' and 'Activity Type' and aggregating data
    summary = df.groupby(['Department', 'Activity Type']).agg({
        'Billable Hours Worked': ['sum', 'mean'],
        'Non Billable Hours': ['sum', 'mean'],
        'Totaal uren': ['sum', 'mean']
    }).reset_index()

    # Renaming columns for clarity
    summary.columns = ['Department', 'Activity Type', 'Total Billable Hours', 'Average Billable Hours',
                       'Total Non Billable Hours', 'Average Non Billable Hours', 'Total Hours', 'Average Hours']

    return summary


category_summary = summarize_categories(df)
category_summary

In [ ]:
# Filter out rows where 'Department' is either 'DevOps' or 'Sales Telecom'
df = df[~df['Department'].isin(['DevOps', 'Sales Telecom'])]

"""
Explanation:
    df['Department'].isin(['DevOps', 'Sales Telecom']) creates a boolean mask where entries are True if the department is 'DevOps' or 'Sales Telecom'.

    The ~ operator negates the boolean mask, so it selects rows that do not match these departments.

    This filtered DataFrame, df_filtered, will contain only the rows where the department is neither 'DevOps' nor 'Sales Telecom'.
"""

In [ ]:
category_summary = summarize_categories(df)
category_summary

In [ ]:
# Step 1: Copy
df_perday = df

# Step 2: Transform the data per day by summing up hours
daily_summary = df_perday.groupby('Work Date').agg({
    'Billable Hours Worked': 'sum',
    'Non Billable Hours': 'sum',
    'Totaal uren': 'sum'
}).reset_index()

daily_summary['Billable Hours Worked'] = daily_summary['Billable Hours Worked'].round(2)
daily_summary['Non Billable Hours'] = daily_summary['Non Billable Hours'].round(2)
daily_summary['Totaal uren'] = daily_summary['Totaal uren'].round(2)

print(daily_summary)

In [ ]:

# Convert 'Work Date' to datetime
df_perday['Work Date'] = pd.to_datetime(df_perday['Work Date'])

# Group by both 'Work Date' and 'Activity Type', then sum 'Totaal uren'
grouped_data = df_perday.groupby([pd.Grouper(key='Work Date', freq='M'), 'Activity Type'])['Totaal uren'].sum().unstack()

# Compute total hours across all activities per month
grouped_data['Total Monthly Hours'] = grouped_data.sum(axis=1)

# Plotting
sns.set(style="whitegrid")
plt.figure(figsize=(14, 7))

# Plot each activity type if available in the data
if 'Ticket' in grouped_data.columns:
    plt.plot(grouped_data.index, grouped_data['Ticket'], label='Ticket Times', marker='o', linestyle='-', color='blue')
if 'Project Task' in grouped_data.columns:
    plt.plot(grouped_data.index, grouped_data['Project Task'], label='Project Times', marker='o', linestyle='-', color='green')
if 'Regular Time' in grouped_data.columns:
    plt.plot(grouped_data.index, grouped_data['Regular Time'], label='Regular Time Entries', marker='o', linestyle='-', color='red')

# Plot total monthly hours
plt.plot(grouped_data.index, grouped_data['Total Monthly Hours'], label='Total Monthly Hours', marker='o', linestyle='-', color='black')

plt.title('Monthly Work Hours Trends by Activity Type')
plt.xlabel('Month')
plt.ylabel('Total Hours')
plt.legend()
plt.grid(True)

# Calculate variance and mean deviation (bias)
variance = grouped_data['Total Monthly Hours'].var()
mean_hours = grouped_data['Total Monthly Hours'].mean()
bias = (grouped_data['Total Monthly Hours'] - mean_hours).mean()

# Annotate variance and bias on the plot
plt.annotate(f'Variance: {variance:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=10, backgroundcolor='white')
plt.annotate(f'Mean Bias: {bias:.2f}', xy=(0.05, 0.90), xycoords='axes fraction', fontsize=10, backgroundcolor='white')

plt.show()

In [ ]:
# Assuming df_perday is loaded with 'Work Date', 'Department', 'Activity Type', and 'Totaal uren'
# Filter for Tickets in IT and Telecom
tickets = df_perday[(df_perday['Activity Type'] == 'Ticket') &
                    df_perday['Department'].isin(['Operations & Service IT', 'Operations & Service Telecom'])]

# Group by Month and Department
monthly_tickets = tickets.groupby([pd.Grouper(key='Work Date', freq='M'), 'Department'])['Totaal uren'].sum().unstack().fillna(0)

# Plotting
sns.set(style="whitegrid")
plt.figure(figsize=(14, 7))
monthly_tickets.plot(kind='line', marker='o', linestyle='-')
plt.title('Monthly Ticket Hours: IT vs. Telecom')
plt.xlabel('Month')
plt.ylabel('Total Ticket Hours')
plt.legend(title='Department')
plt.grid(True)
plt.show()

Voorspelling voor Operations & Service IT projecten

In [ ]:
# Filter for 'Operations & Service IT' department and 'Project Task' activity type
filtered_df = df[(df['Department'] == 'Operations & Service IT') & (df['Activity Type'] == 'Project Task')]

# Group by 'Work Date' and sum 'Totaal uren'
daily_summary = filtered_df.groupby('Work Date').agg({
    'Totaal uren': 'sum'
}).reset_index()

# Set 'Work Date' as the index
daily_data = daily_summary.set_index('Work Date')['Totaal uren']
daily_data
filtered_df = df[(df['Department'] == 'Operations & Service IT') & (df['Activity Type'] == 'Project Task')]

# Group by month and sum 'Totaal uren'
monthly_summary = filtered_df.groupby(pd.Grouper(key='Work Date', freq='M')).agg({
    'Totaal uren': 'sum'
}).reset_index()

# Set 'Work Date' as the index
monthly_data = monthly_summary.set_index('Work Date')['Totaal uren']
monthly_data

In [ ]:
#Base model
# Persistence Model
months_to_forecast = 12  # Forecasting for the next 12 months
X = monthly_data.values
train, test = X[:-months_to_forecast], X[-months_to_forecast:]
predictions = np.roll(test, 1)
predictions[0] = train[-1]  # Set the first prediction to the last value of the train set

# Calculate RMSE for the Persistence Model
persistence_rmse = np.sqrt(mean_squared_error(test, predictions))
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')

In [ ]:
# Step 2: Check for Stationarity
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    result = adfuller(timeseries)
    dfoutput = pd.Series(result[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in result[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

test_stationarity(monthly_data)

Data is niet stationare

In [ ]:
# Step 3: Plot ACF and PACF
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(monthly_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(monthly_data, lags=10, ax=ax[1])
plt.show()

In [ ]:
# Step 4: Decompose the Time Series
decomposition = sm.tsa.seasonal_decompose(monthly_data, model='additive')
fig = decomposition.plot()
plt.show()

Laat duidelijke seasonal trend zien

In [ ]:
differenced_data = monthly_data.diff().dropna()
test_stationarity(differenced_data)
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(differenced_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(differenced_data, lags=10, ax=ax[1])
plt.show()

Na differencing is de data is getransformeerd naar een stationaire reeks, wel met verminderde/verdwenen autocorrelatie.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Prepare the data for linear regression
time = np.arange(len(monthly_data)).reshape(-1, 1)
values = monthly_data.values.reshape(-1, 1)

# Fit linear regression
model = LinearRegression()
model.fit(time, values)
trend = model.predict(time)

# Detrend the data
detrended_data = monthly_data - trend.flatten()

# Check stationarity of detrended data
test_stationarity(detrended_data)

# Re-plot ACF and PACF for detrended data
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(detrended_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(detrended_data, lags=10, ax=ax[1])
plt.show()


Differencing is prefered over linear detrending for making the data stationar.
When differencing the ACF and PACF plots show significant autocorrelation at lag 1, and the values taper off quickly after that. This indicates that differencing has removed much of the autocorrelation in the data. However,given the drop at lag 6 in both the ACF and PACF plots, it's possible that some seasonal pattern remains in the data even after differencing.

In [ ]:
#SARIMA
import warnings
warnings.filterwarnings("ignore")
tst_size = 23
trn, tst = monthly_data[:-tst_size], monthly_data[-tst_size:]
best_aic=float('inf')
best_bic=float('inf')
aic_settings=()
bic_settings=()

for p,d,q in itertools.product([ 1, 2, 3],[1, 2],[1, 2]):
    model = sm.tsa.SARIMAX(trn, order=(p, d, q))
    try:
        model_fit = model.fit()
        aic = model_fit.aic
        bic = model_fit.bic

        if aic < best_aic:
            best_aic = aic
            aic_settings = (p, d, q)

        if bic < best_bic:
            best_bic = bic
            bic_settings = (p, d, q)

    except:
        continue
print("Best AIC:", best_aic)
print("AIC Settings:", aic_settings)
print("Best BIC:", best_bic)
print("BIC Settings:", bic_settings)

# Performance
model = tsa.ARIMA(trn, order=(23, 2, 1))
model_fit = model.fit()
prd = model_fit.predict(start=len(trn), end=len(trn)+len(tst)-1)
plt.plot(tst,'.-')
plt.plot(prd, '--',color='red')
plt.show()

In [ ]:
# SARIMA met validatie set
tst_size = 12  # Number of test periods
val_size = 12  # Number of validation periods

trn = monthly_data[:-tst_size - val_size]
val = monthly_data[-tst_size - val_size:-tst_size]
tst = monthly_data[-tst_size:]

best_rmse = float('inf')
best_settings = ()

# Grid search for SARIMA parameters
for p, d, q in itertools.product(range(0, 3), range(0, 3), range(0, 3)):
    for P, D, Q, m in itertools.product(range(0, 3), range(0, 3), range(0, 3), [12]):
        try:
            model = SARIMAX(trn, order=(p, d, q), seasonal_order=(P, D, Q, m))
            model_fit = model.fit(disp=False)
            val_predictions = model_fit.get_forecast(len(val)).predicted_mean
            rmse = np.sqrt(mean_squared_error(val, val_predictions))

            if rmse < best_rmse:
                best_rmse = rmse
                best_settings = ((p, d, q), (P, D, Q, m))

        except:
            continue

print("Best settings:", best_settings)

# Performance on the test set
model = SARIMAX(monthly_data, order=best_settings[0], seasonal_order=best_settings[1])
model_fit = model.fit(disp=False)

# Forecasting
forecast_steps = len(tst)
prd = model_fit.get_forecast(steps=forecast_steps).predicted_mean
forecast_dates = pd.date_range(start=tst.index[0], periods=forecast_steps, freq='M')
forecast_df = pd.DataFrame({'Forecast': prd.values}, index=forecast_dates)

# RMSE for the test set
test_rmse = np.sqrt(mean_squared_error(tst, prd))
print(f'Test RMSE: {test_rmse:.3f}')

# Plotting the actual vs forecasted values
plt.figure(figsize=(12, 6))
plt.plot(monthly_data.index, monthly_data, label='Actual')
plt.plot(forecast_df.index, forecast_df['Forecast'], label='Forecast', linestyle='--', color='red')
plt.title('SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

**Subconclusion:**when choosing the appropriate aggregation level for time series analysis, it's important to balance business needs and the robustness of the model. For strategic purposes, where the goal is to identify long-term trends and seasonality, monthly aggregation is typically sufficient and commonly used. However, due to the limited number of data points currently available in the dataset and low accurcy when plotting for a monthly scope, weekly aggregation is advised. Weekly aggregation strikes a balance between granularity and noise reduction, providing more data points for meaningful analysis while capturing important patterns and trends. This approach enhances the accuracy of the forecasts and aligns with both business requirements and machine learning practices. As more data points become available, it can be reconsidered transitioning to monthly aggregation for strategic analysis.

In [ ]:
# Aggregate de data bij week, summing 'Totaal uren'
df = pd.DataFrame(df)

df['Work Date'] = pd.to_datetime(df['Work Date'])

print(df.dtypes)

weekly_summary = df.resample('W-Sun', on='Work Date').sum().reset_index().sort_values('Work Date')
weekly_summary.head()

weekly_data = weekly_summary.set_index('Work Date')['Totaal uren']
print(weekly_data.head())

In [ ]:
# Predictie scope is week
weekly_summary = filtered_df.groupby(pd.Grouper(key='Work Date', freq='W')).agg({'Totaal uren': 'sum'}).reset_index()
weekly_data = weekly_summary.set_index('Work Date')['Totaal uren']

# Step 1: Check for Stationarity
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    result = adfuller(timeseries)
    dfoutput = pd.Series(result[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in result[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

test_stationarity(weekly_data)

# Step 2: Plot ACF and PACF
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(weekly_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(weekly_data, lags=10, ax=ax[1])
plt.show()

# Step 3: Decompose the Time Series
decomposition = sm.tsa.seasonal_decompose(weekly_data, model='additive')
fig = decomposition.plot()
plt.show()

# Step 4: Difference the data and re-check for stationarity
differenced_data = weekly_data.diff().dropna()
test_stationarity(differenced_data)

# Step 5: Plot ACF and PACF for differenced data
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(differenced_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(differenced_data, lags=10, ax=ax[1])
plt.show()

Deze analyse van de dataset op week scope laat zien dat, na het differentiëren, stationariteit is bereikt en significante autocorrelaties aanwezig blijven. Dit zijn positieve signalen die erop wijzen dat tijdreeksmodellen effectief onderliggende patronen in de gegevens zou kunnen vastleggen en voorspellen.

In [ ]:
weekly_data.describe()
weekly_data.head()

In [ ]:
# Prediction scope op week
# Step 1: Check for Stationarity
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    result = adfuller(timeseries)
    dfoutput = pd.Series(result[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in result[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

test_stationarity(weekly_data)

# Step 2: Plot ACF and PACF
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(weekly_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(weekly_data, lags=10, ax=ax[1])
plt.show()

# Step 3: Decompose the Time Series
decomposition = sm.tsa.seasonal_decompose(weekly_data, model='additive', period=52)  # Assuming 52 weeks seasonality
fig = decomposition.plot()
plt.show()

# Step 4: Difference the data and re-check for stationarity
differenced_data = weekly_data.diff().dropna()
test_stationarity(differenced_data)

# Step 5: Plot ACF and PACF for differenced data
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(differenced_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(differenced_data, lags=10, ax=ax[1])
plt.show()

# Persistence Model
weeks_to_forecast = 12  # Forecasting for the next 12 weeks
X = weekly_data.values
train, test = X[:-weeks_to_forecast], X[-weeks_to_forecast:]
predictions = np.roll(test, 1)
predictions[0] = train[-1]  # Set the first prediction to the last value of the train set

# Create a DataFrame for the persistence model
forecast_dates = weekly_data.index[-weeks_to_forecast:]
persistence_forecast = pd.DataFrame({'Actual': weekly_data[-weeks_to_forecast:], 'Persistence': predictions}, index=forecast_dates)

# Calculate RMSE for the Persistence Model
persistence_rmse = np.sqrt(mean_squared_error(test, predictions))
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')

# Step 6: Identify the best SARIMA model parameters using grid search
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in pdq]  # Using 52 for yearly seasonality in weekly data

best_aic = float("inf")
best_pdq = None
best_seasonal_pdq = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(weekly_data, order=param, seasonal_order=param_seasonal)
            results = mod.fit(disp=False)
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            continue

print(f"Best SARIMA order: {best_pdq}")
print(f"Best seasonal order: {best_seasonal_pdq}")

# Step 7: Fit the SARIMA model with parameters
mod = SARIMAX(weekly_data, order=best_pdq, seasonal_order=best_seasonal_pdq)
results = mod.fit(disp=False)

# Step 8: Forecast future values
forecast_steps = weeks_to_forecast
forecast = results.get_forecast(steps=forecast_steps)
sarima_pred = forecast.predicted_mean
sarima_pred.index = forecast_dates  # Ensure the SARIMA predictions have the same index

# Create a DataFrame for the SARIMA model
sarima_forecast = pd.DataFrame({'SARIMA': sarima_pred}, index=forecast_dates)

# Calculate RMSE for the SARIMA Model
sarima_rmse = np.sqrt(mean_squared_error(test, sarima_pred))
print(f'SARIMA Model Test RMSE: {sarima_rmse:.3f}')

# Combine the actual values, persistence model predictions, and SARIMA predictions
combined_forecast = pd.concat([persistence_forecast, sarima_forecast], axis=1)

# Step 9: Plot
combined_forecast.plot(figsize=(12, 6))
plt.title('Actual vs Persistence Model vs SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

# Step 10: Evaluate the model performance on the test set (already done above)
# Persistence RMSE
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')
# SARIMA RMSE
print(f'SARIMA Model Test RMSE: {sarima_rmse:.3f}')

ACF en PACF plots (na en voor  differencing) bevstigen dat de dataset een sterke tijd component heeft, wat tijd serie analyse bruikbaar maakt.

# **Voorspellen totaal uren projecten en tasks 	Operations & Service Telecom en 	Operations & Service IT**

In [ ]:
#Week scope
# Step 1: Check for Stationarity
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    result = adfuller(timeseries)
    dfoutput = pd.Series(result[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in result[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

test_stationarity(weekly_data)

# Step 2: Plot ACF and PACF
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(weekly_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(weekly_data, lags=10, ax=ax[1])
plt.show()

# Step 3: Decompose the Time Series
decomposition = sm.tsa.seasonal_decompose(weekly_data, model='additive', period=52)  # Assuming 52 weeks seasonality
fig = decomposition.plot()
plt.show()

# Step 4: Difference the data and re-check for stationarity
differenced_data = weekly_data.diff().dropna()
test_stationarity(differenced_data)

# Step 5: Plot ACF and PACF for differenced data
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sm.graphics.tsa.plot_acf(differenced_data, lags=10, ax=ax[0])
sm.graphics.tsa.plot_pacf(differenced_data, lags=10, ax=ax[1])
plt.show()

# Persistence Model
weeks_to_forecast = 26  # Forecasting for the next 26 weeks (approximately 6 months)
X = weekly_data.values
train, test = X[:-weeks_to_forecast], X[-weeks_to_forecast:]
predictions = np.roll(test, 1)
predictions[0] = train[-1]  # Set the first prediction to the last value of the train set

# Create a DataFrame for the persistence model
forecast_dates = weekly_data.index[-weeks_to_forecast:]
persistence_forecast = pd.DataFrame({'Actual': weekly_data[-weeks_to_forecast:], 'Persistence': predictions}, index=forecast_dates)

# Calculate RMSE for the Persistence Model
persistence_rmse = np.sqrt(mean_squared_error(test, predictions))
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')

# Split the dataset into training, validation, and test sets
total_weeks = len(weekly_data)
train_size = int(0.7 * total_weeks)
val_size = int(0.15 * total_weeks)
test_size = total_weeks - train_size - val_size

trn = weekly_data[:train_size]
val = weekly_data[train_size:train_size + val_size]
tst = weekly_data[train_size + val_size:]

best_rmse = float('inf')
best_settings = ()

# Step 6: Grid search for SARIMA parameters using validation set
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in pdq]  # Using 52 for yearly seasonality in weekly data

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            model = SARIMAX(trn, order=param, seasonal_order=param_seasonal)
            model_fit = model.fit(disp=False)
            val_predictions = model_fit.get_forecast(steps=len(val)).predicted_mean
            rmse = np.sqrt(mean_squared_error(val, val_predictions))

            if rmse < best_rmse:
                best_rmse = rmse
                best_settings = (param, param_seasonal)

        except:
            continue

print(f"Best settings: {best_settings}")

# Step 7: Fit the SARIMA model with  parameters
model = SARIMAX(weekly_data[:train_size + val_size], order=best_settings[0], seasonal_order=best_settings[1])
model_fit = model.fit(disp=False)

# Step 8: Forecast future values
forecast_steps = test_size
forecast = model_fit.get_forecast(steps=forecast_steps)
sarima_pred = forecast.predicted_mean
sarima_pred.index = weekly_data.index[train_size + val_size:train_size + val_size + forecast_steps]

sarima_forecast = pd.DataFrame({'SARIMA': sarima_pred}, index=sarima_pred.index)

# Calculate RMSE for the SARIMA Model
sarima_rmse = np.sqrt(mean_squared_error(tst, sarima_pred))
print(f'SARIMA Model Test RMSE: {sarima_rmse:.3f}')

# Actual values, persistence model predictions, and SARIMA predictions
combined_forecast = pd.concat([persistence_forecast, sarima_forecast], axis=1)

# Step 9: Plot
combined_forecast.plot(figsize=(12, 6))
plt.title('Actual vs Persistence Model vs SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

# Step 10: Evaluate the model performance on the test set (already done above)
# Persistence RMSE
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')
# SARIMA RMSE
print(f'SARIMA Model Test RMSE: {sarima_rmse:.3f}')


Ook voor 	Operations & Service Telecom en 	Operations & Service IT tickets en projecten dataset, laat een sterke stijgende trend over de jaren zien en een seasonal effect. Na het differentiëren, stationariteit is bereikt en significante autocorrelaties aanwezig blijven en dus een tijdcomponent. Dit zijn positieve signalen die erop wijzen dat tijdreeksmodellen effectief onderliggende patronen in de gegevens zou kunnen vastleggen en voorspellen.

In [ ]:
# SARIMA Model
warnings.filterwarnings("ignore")
best_aic = float('inf')
best_bic = float('inf')
aic_settings = ()
bic_settings = ()

# Grid search for SARIMA parameters
for p, d, q in itertools.product([2, 3, 4, 6, 7, 8], [1, 2], [1, 2]):
    for P, D, Q, m in itertools.product(range(0, 3), range(0, 3), range(0, 3), [52]):
        try:
            model = SARIMAX(trn, order=(p, d, q), seasonal_order=(P, D, Q, m))
            model_fit = model.fit(disp=False)
            aic = model_fit.aic
            bic = model_fit.bic

            if aic < best_aic:
                best_aic = aic
                aic_settings = (p, d, q, P, D, Q, m)

            if bic < best_bic:
                best_bic = bic
                bic_settings = (p, d, q, P, D, Q, m)

        except:
            continue

print("Best AIC:", best_aic)
print("AIC Settings:", aic_settings)
print("Best BIC:", best_bic)
print("BIC Settings:", bic_settings)

# Fit the SARIMA model with parameters
p, d, q, P, D, Q, m = aic_settings
model = SARIMAX(weekly_data[:train_size + val_size], order=(p, d, q), seasonal_order=(P, D, Q, m))
model_fit = model.fit(disp=False)

# Forecast future values
forecast_steps = test_size
forecast = model_fit.get_forecast(steps=forecast_steps)
sarima_pred = forecast.predicted_mean
sarima_pred.index = weekly_data.index[train_size + val_size:train_size + val_size + forecast_steps]

#RMSE, MAE, and MAPE for the SARIMA Model
sarima_rmse = np.sqrt(mean_squared_error(tst, sarima_pred))
sarima_mae = mean_absolute_error(tst, sarima_pred)
sarima_mape = mean_absolute_percentage_error(tst, sarima_pred)
print(f'SARIMA Model Test RMSE: {sarima_rmse:.3f}')
print(f'SARIMA Model Test MAE: {sarima_mae:.3f}')
print(f'SARIMA Model Test MAPE: {sarima_mape:.3f}')

combined_forecast = pd.concat([forecast_df, pd.DataFrame({'Prophet': prophet_pred}, index=prophet_pred.index), pd.DataFrame({'SARIMA': sarima_pred}, index=sarima_pred.index)], axis=1)

combined_forecast.plot(figsize=(12, 6))
plt.title('Actual vs Persistence Model vs Prophet Model vs SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

In [ ]:
df['Work Date'] = pd.to_datetime(df['Work Date'])
daily_summary = df.groupby('Work Date').agg({'Totaal uren': 'sum'}).reset_index()
daily_summary = daily_summary[daily_summary['Totaal uren'] >= 50]
daily_data = daily_summary.set_index('Work Date')['Totaal uren']
daily_data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
import itertools
import warnings

warnings.filterwarnings("ignore")

# Training, validation, and test sets
total_days = len(daily_data)
train_size = int(0.7 * total_days)
val_size = int(0.15 * total_days)
test_size = total_days - train_size - val_size

trn = daily_data[:train_size]
val = daily_data[train_size:train_size + val_size]
tst = daily_data[train_size + val_size:]

# Persistence Model
days_to_forecast = test_size
X = daily_data.values
train, test = X[:-days_to_forecast], X[-days_to_forecast:]
predictions = np.roll(test, 1)
predictions[0] = train[-1]  # Set the first prediction to the last value of the train set

forecast_dates = daily_data.index[-days_to_forecast:]
persistence_forecast = pd.DataFrame({'Actual': daily_data[-days_to_forecast:], 'Persistence': predictions}, index=forecast_dates)

#RMSE for the Persistence Model
persistence_rmse = np.sqrt(mean_squared_error(test, predictions))
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')

# Grid search for SARIMA parameters
best_rmse = float('inf')
best_settings = ()

for p, d, q in itertools.product(range(0, 3), range(0, 3), range(0, 3)):
    for P, D, Q, m in itertools.product(range(0, 3), range(0, 3), range(0, 3), [12]):
        try:
            model = SARIMAX(trn, order=(p, d, q), seasonal_order=(P, D, Q, m))
            model_fit = model.fit(disp=False)
            val_predictions = model_fit.get_forecast(len(val)).predicted_mean
            rmse = np.sqrt(mean_squared_error(val, val_predictions))

            if rmse < best_rmse:
                best_rmse = rmse
                best_settings = ((p, d, q), (P, D, Q, m))

        except:
            continue

print("Best settings:", best_settings)

# Fit the SARIMA model with the identified parameters
model = SARIMAX(daily_data[:train_size + val_size], order=best_settings[0], seasonal_order=best_settings[1])
model_fit = model.fit(disp=False)

# Forecast future values
forecast_steps = test_size
forecast = model_fit.get_forecast(steps=forecast_steps)
sarima_pred = forecast.predicted_mean
sarima_pred.index = daily_data.index[train_size + val_size:train_size + val_size + forecast_steps]

# Calculate Forecast Errors
sarima_errors = tst - sarima_pred

# Calculate RMSE and MFE for Bias
sarima_rmse = np.sqrt(mean_squared_error(tst, sarima_pred))
sarima_mfe = np.mean(sarima_errors)
print(f'SARIMA Model Test RMSE: {sarima_rmse:.3f}')
print(f'SARIMA Model Mean Forecast Error (Bias): {sarima_mfe:.3f}')

# Combine the actual values, persistence model predictions, and SARIMA predictions into one DataFrame
combined_forecast = pd.concat([persistence_forecast, pd.DataFrame({'SARIMA': sarima_pred}, index=sarima_pred.index)], axis=1)

# Plot the combined forecast DataFrame
combined_forecast.plot(figsize=(12, 6))
plt.title('Actual vs Persistence Model vs SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

# Plot Forecast Errors
plt.figure(figsize=(12, 6))
plt.plot(sarima_errors, label='SARIMA Forecast Errors')
plt.axhline(y=0, color='r', linestyle='--')
plt.title('SARIMA Model Forecast Errors')
plt.xlabel('Date')
plt.ylabel('Error')
plt.legend()
plt.show()

# Plot Actual vs Forecast
plt.figure(figsize=(12, 6))
plt.plot(tst, label='Actual')
plt.plot(sarima_pred, label='SARIMA Forecast')
plt.title('Actual vs SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()


De lagere RMSE van het persistence model suggereert dat de total hours een zekere mate van continuïteit en lage variabiliteit vertonen, waardoor eenvoudige voorspellingen op basis van recente gegevens effectief zijn.

Prophet modelling

In [ ]:
#Week scope
!pip install Prophet
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
# Training, validation, and test sets
total_weeks = len(weekly_data)
train_size = int(0.7 * total_weeks)
val_size = int(0.15 * total_weeks)
test_size = total_weeks - train_size - val_size

trn = weekly_data[:train_size]
val = weekly_data[train_size:train_size + val_size]
tst = weekly_data[train_size + val_size:]

# Prophet Model
prophet_df = weekly_summary.rename(columns={'Work Date': 'ds', 'Totaal uren': 'y'})
prophet_train = prophet_df[:train_size + val_size]

model_prophet = Prophet()
model_prophet.fit(prophet_train)
future = model_prophet.make_future_dataframe(periods=test_size, freq='W')
forecast_prophet = model_prophet.predict(future)

# Extract the test period forecast
prophet_pred = forecast_prophet.set_index('ds')['yhat'].iloc[train_size + val_size:]

# Calculate RMSE for the Prophet Model
prophet_rmse = np.sqrt(mean_squared_error(tst, prophet_pred))
print(f'Prophet Model Test RMSE: {prophet_rmse:.3f}')

In [ ]:
combined_forecast = pd.DataFrame({'Actual': tst, 'Prophet': prophet_pred}, index=tst.index)

# Plot
combined_forecast.plot(figsize=(12, 6))
plt.title('Actual vs Prophet Model')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

In [ ]:
prophet_errors = tst - prophet_pred

# Calculate Mean Forecast Error (MFE) for Bias
prophet_mfe = np.mean(prophet_errors)
print(f'Prophet Model Mean Forecast Error (Bias): {prophet_mfe:.3f}')

In [ ]:
total_weeks = len(weekly_data)
train_size = int(0.7 * total_weeks)
val_size = int(0.15 * total_weeks)
test_size = total_weeks - train_size - val_size

trn = weekly_data[:train_size]
val = weekly_data[train_size:train_size + val_size]
tst = weekly_data[train_size + val_size:]
test_weekly_average = weekly_data.loc[tst.index].mean()
normalized_weekly_rmse = prophet_rmse / test_weekly_average

# Print the normalized weekly RMSE
print(f'Normalized Weekly RMSE: {normalized_weekly_rmse}')

In [ ]:
rmse = 101.545
n = len(tst)

# Calculate the Standard Error
se = rmse / np.sqrt(n)

# Confidence interval multiplier for 95% CI
z_score = 1.96

# Calculate the Margin of Error
moe = z_score * se
rmse_percentage = (rmse / test_weekly_average) * 100
print(f'Average Daily Total Hours: {test_weekly_average:.2f}')
print(f'RMSE as Percentage of Average Daily Total Hours: {rmse_percentage:.2f}%')

In [ ]:

# Persistence Model
weeks_to_forecast = test_size
X = weekly_data.values
train, test = X[:-weeks_to_forecast], X[-weeks_to_forecast:]
predictions = np.roll(test, 1)
predictions[0] = train[-1]  # Set the first prediction to the last value of the train set

#RMSE for the Persistence Model
persistence_rmse = np.sqrt(mean_squared_error(test, predictions))
print(f'Persistence Model Test RMSE: {persistence_rmse:.3f}')

forecast_dates = weekly_data.index[-weeks_to_forecast:]
forecast_df = pd.DataFrame({'Actual': test, 'Prediction': predictions}, index=forecast_dates)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(weekly_data.index, weekly_data, label='Actual')
plt.plot(forecast_df.index, forecast_df['Prediction'], label='Persistence Forecast', linestyle='--')
plt.title('Actual vs Persistence Model Forecast (Weekly)')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()
prophet_df = weekly_summary.rename(columns={'Work Date': 'ds', 'Totaal uren': 'y'})
prophet_train = prophet_df[:train_size + val_size]

model_prophet = Prophet()
model_prophet.fit(prophet_train)
future = model_prophet.make_future_dataframe(periods=test_size, freq='W')
forecast_prophet = model_prophet.predict(future)

prophet_pred = forecast_prophet.set_index('ds')['yhat'].iloc[train_size + val_size:]

#RMSE for the Prophet Model
prophet_rmse = np.sqrt(mean_squared_error(tst, prophet_pred))
print(f'Prophet Model Test RMSE: {prophet_rmse:.3f}')

# Plot
plt.figure(figsize=(12, 6))
plt.plot(weekly_data.index, weekly_data, label='Actual')
plt.plot(forecast_df.index, forecast_df['Prediction'], label='Persistence Forecast', linestyle='--')
plt.plot(prophet_pred.index, prophet_pred, label='Prophet Forecast', linestyle='--')
plt.title('Actual vs Prophet Model vs Persistence Model Forecast (Weekly)')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

In [ ]:
#Op dag
!pip install Prophet
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
# Aggregate the data by day, summing 'Totaal uren'
daily_summary = df.groupby('Work Date').agg({'Totaal uren': 'sum'}).reset_index()
print(daily_summary)

# Filter out days with less than 50 total hours
daily_summary = daily_summary[daily_summary['Totaal uren'] >= 50]

# Set 'Work Date' as the index
daily_data = daily_summary.set_index('Work Date')['Totaal uren']
print(daily_data.head())

# Split the dataset into training, validation, and test sets
total_days = len(daily_data)
train_size = int(0.7 * total_days)
val_size = int(0.15 * total_days)
test_size = total_days - train_size - val_size

trn = daily_data[:train_size]
val = daily_data[train_size:train_size + val_size]
tst = daily_data[train_size + val_size:]

# Prophet Model
prophet_df = daily_summary.rename(columns={'Work Date': 'ds', 'Totaal uren': 'y'})
prophet_train = prophet_df[:train_size + val_size]

model_prophet = Prophet()
model_prophet.fit(prophet_train)

# Extend the future dataframe to 365 days for a full year prediction
future = model_prophet.make_future_dataframe(periods=1095, freq='D')
forecast_prophet = model_prophet.predict(future)

# Extract the test period forecast
prophet_pred = forecast_prophet.set_index('ds')['yhat'].iloc[train_size + val_size:total_days + 1095]

# Calculate RMSE for the Prophet Model on the test set
prophet_rmse = np.sqrt(mean_squared_error(tst, prophet_pred[:test_size]))
print(f'Prophet Model Test RMSE: {prophet_rmse:.3f}')

# Combine the actual values and predictions into one DataFrame
combined_forecast = pd.DataFrame({'Actual': tst, 'Prophet': prophet_pred[:test_size]}, index=tst.index)

# Plot the actual vs forecasted data
plt.figure(figsize=(12, 6))
plt.plot(combined_forecast.index, combined_forecast['Actual'], label='Actual')
plt.plot(combined_forecast.index, combined_forecast['Prophet'], label='Prophet Forecast')
plt.title('Actual vs Prophet Model')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

# Plot the future forecast
plt.figure(figsize=(12, 6))
plt.plot(daily_data.index, daily_data, label='Historical Data')
plt.plot(forecast_prophet['ds'], forecast_prophet['yhat'], label='Prophet Forecast')
plt.title('Prophet Forecast for Next Year')
plt.xlabel('Date')
plt.ylabel('Total Hours')
plt.legend()
plt.show()

In [ ]:
daily_data.head()

In [ ]:
prophet_errors = tst - prophet_pred

# Calculate Mean Forecast Error (MFE) for Bias
prophet_mfe = np.mean(prophet_errors)
print(f'Prophet Model Mean Forecast Error (Bias): {prophet_mfe:.3f}')

Negative bais dus tendency to overpredict

Prophet heeft hogere accurcy dan SARIMA. Prophet heeft de neiging te overpredictne en SARIMA te onderpredicten. Op dit moment functioneer het base model nog steeds het best.

In [ ]:
total_days = len(daily_data)
train_size = int(0.7 * total_days)
val_size = int(0.15 * total_days)
test_size = total_days - train_size - val_size

trn = daily_data[:train_size]
val = daily_data[train_size:train_size + val_size]
tst = daily_data[train_size + val_size:]
test_daily_average = daily_data.loc[tst.index].mean()

# Normalize the RMSE with the daily average
normalized_daily_rmse = prophet_rmse / test_daily_average

# Print the normalized daily RMSE
print(f'Normalized Daily RMSE: {normalized_daily_rmse}')

In [ ]:
test_daily_average

In [ ]:
rmse = 31.579
n = len(tst)

# Calculate the Standard Error
se = rmse / np.sqrt(n)

# Confidence interval multiplier for 95% CI
z_score = 1.96

# Calculate the Margin of Error
moe = z_score * se
rmse_percentage = (rmse / test_daily_average) * 100
print(f'Average Daily Total Hours: {test_daily_average:.2f}')
print(f'RMSE as Percentage of Average Daily Total Hours: {rmse_percentage:.2f}%')